In [1]:
import pandas as pd
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
%matplotlib inline

Using TensorFlow backend.
/opt/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/envs/tensorflow/l

In [2]:
filename = r'Womens Clothing E-Commerce Reviews.csv'

WomensClothing= pd.read_csv(filename,encoding='latin-1')
WomensClothing.head(5)

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [3]:
WomensClothing['Review Text'] = WomensClothing['Review Text'].astype(str)
train, test = train_test_split(WomensClothing, test_size=0.20, random_state=2019, stratify = WomensClothing[['Rating']])
labels = train['Rating']
str(train)

'       Clothing ID  Age                              Title  \\\n12302          509   69  Juneberry sleep pants perfect fit   \n13962         1098   31   High quality one of a kind piece   \n5731           868   30                                NaN   \n10224          147   46                                NaN   \n7366           965   27              Cute but poor quality   \n...            ...  ...                                ...   \n22578         1070   58                           Perfect!   \n18634         1072   36                          Shape odd   \n18800         1047   53                                NaN   \n16841         1104   29             Buy if you have no fat   \n18484         1080   39                     Gorgeous print   \n\n                                             Review Text  Rating  \\\n12302  I love these sleep pants they are the perfect ...       5   \n13962  This dress had my attention immediately online...       5   \n5731   Great shirt, more of a st

In [4]:
vocab_size = 10000
encoded_docs=[]

for index, row in train.iterrows():
    if index % 10000 ==0:
        print(index)
    text = row['Review Text']
    encoded_docs.append(one_hot(text, vocab_size))

0
20000


In [5]:
max_length = len(max(encoded_docs,key=len))
max_length

padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)
len(padded_docs)

[[1262 7890 2444 ...    0    0    0]
 [ 998 6161 2689 ...    0    0    0]
 [3857 7395  502 ...    0    0    0]
 ...
 [9632 2444 7364 ...    0    0    0]
 [ 409 1262 9513 ...    0    0    0]
 [1262  237  832 ...    0    0    0]]


18788

In [6]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=max_length))
model.add(Flatten())
model.add(Dense(6, activation='softmax'))
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['acc'])
model.summary()

model.fit(padded_docs, labels, epochs=10, verbose=2)






_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 116, 50)           500000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 5800)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 34806     
Total params: 534,806
Trainable params: 534,806
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/10
 - 4s - loss: 1.0535 - acc: 0.5863
Epoch 2/10
 - 4s - loss: 0.7825 - acc: 0.6836
Epoch 3/10
 - 4s - loss: 0.5838 - acc: 0.7966
Epoch 4/10
 - 4s - loss: 0.4007 - acc: 0.8841
Epoch 5/10
 - 4s - loss: 0.2705 - acc: 0.9341
Epoch 6/10
 - 4s - loss: 0.1872 - acc: 0.9602
Epoch 7/10
 - 4s - loss:

In [7]:
encoded_docs_test=[]
for index, row in test.iterrows():
    if index % 10000 ==0:
        print(index)
    text = row['Review Text']
    encoded_docs_test.append(one_hot(text, vocab_size))

10000


In [8]:
padded_docs_test = pad_sequences(encoded_docs_test, maxlen=max_length, padding='post')
print(padded_docs_test)

labels_test = test['Rating']
loss, accuracy = model.evaluate(padded_docs_test, labels_test, verbose=2)
print('Accuracy: %f' % (accuracy*100))

[[3857 6716 8953 ...    0    0    0]
 [9695 8550 1499 ...    0    0    0]
 [1262 7550  998 ...    0    0    0]
 ...
 [ 998 9914 3522 ...    0    0    0]
 [2444 3728  502 ...    0    0    0]
 [9695 3522 3671 ...    0    0    0]]
Accuracy: 60.323542
